In [25]:
import matplotlib.pyplot as plt
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
#import soundfile as sf
import os
import glob
import numpy as np
from PIL import Image
from scipy.fftpack import fft
from scipy.misc import imresize
from IPython.display import Image
import mobilenet_model as mobilenet_v1
from IPython.core.display import HTML
from tensorflow.contrib.framework.python.ops import arg_scope
import numpy as np
import pandas as pd
from sklearn import metrics
from absl import flags
import tensorflow as tf
import operator
from PIL import Image
import seaborn as sns
import cv2
import random
import tensorflow_hub as hub
from tensorflow.python.data import Dataset
%matplotlib inline
# tf.logging.set_verbosity(tf.logging.INFO)
tf.logging.set_verbosity(tf.logging.ERROR)

In [26]:
test_pict_path = './input/picts/test/'

In [27]:
labels_to_keep = ['yes', 'no', 'up', 'down', 'left',
                  'right', 'on', 'off', 'stop', 'go']

test_labels = ['yes', 'no', 'up', 'down', 'left',
                  'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']

label_dict = {'yes':0, 'no':1, 'up':2, 'down':3, 'left':4,
                  'right':5, 'on':6, 'off':7, 'stop':8, 'go':9, 'silence':10, 'unknown':11}

inverted_dict = dict([[v,k] for k,v in label_dict.items()])
inverted_dict

{0: 'yes',
 1: 'no',
 2: 'up',
 3: 'down',
 4: 'left',
 5: 'right',
 6: 'on',
 7: 'off',
 8: 'stop',
 9: 'go',
 10: 'silence',
 11: 'unknown'}

In [30]:
all_files = [y for y in os.listdir(test_pict_path) if '.jpg' in y]
for x in range(0,15):
    test_images = []
    fnames = []
    x *= 10000
    for file in all_files[:x]:
        test_images.append(cv2.imread(test_pict_path + file))
        fnames.append(file[:-4] + '.wav')
    test_images = np.array(test_images, dtype="float32")
    np.save('all_array_' + str(x), test_images)
    np.save('all_labels_' + str(x), fnames)

MemoryError: 

In [7]:
def mobilenet_model_fn(features, labels, mode):
    # Load Inception-v3 model.
    
    module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_035_96/feature_vector/2")
    input_layer = features
    outputs = module(input_layer)
    logits = tf.layers.dense(inputs=outputs, units=12) #
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [14]:
classifier = tf.estimator.Estimator(
    model_fn=mobilenet_model_fn,
    model_dir="./all_model",
    )

In [23]:
def my_test_input_fn(features, batch_size=1, shuffle=True, num_epochs=None):
    raw_features = features
    ds = Dataset.from_tensor_slices((raw_features))
    ds = ds.batch(batch_size).repeat(num_epochs)
    if shuffle:
        ds = ds.shuffle(10000)
    features = ds.make_one_shot_iterator().get_next()
    return features

predict_test_input_fn = lambda: my_test_input_fn(
    test_images,
    num_epochs=1,
    shuffle=False)

test_predictions = classifier.predict(input_fn=predict_test_input_fn)
test_predictions = np.array([item['classes'] for item in test_predictions])
test_predictions = list(map(inverted_dict.get, test_predictions))
print(test_predictions)

['right', 'off', 'up', 'off', 'on', 'up', 'down', 'off', 'no', 'on', 'silence', 'silence', 'left', 'silence', 'no', 'right', 'down', 'on', 'on', 'on', 'stop', 'down', 'on', 'on', 'down', 'off', 'stop', 'stop', 'right', 'silence', 'silence', 'left', 'down', 'stop', 'silence', 'stop', 'off', 'down', 'on', 'off']


In [24]:
my_submission = pd.DataFrame({'fname': fnames, 'label': test_predictions})
# you could use any filename. We choose submission here
my_submission.to_csv('sub_voice_tpu.csv', index=False)